In [ ]:
#Import Packages
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
import pandas as pd

In [ ]:
#Initialize Variables
alphabet = ['a','b','c','d','e','f','g','h','i','j',
            'k','l','m','n','o','p','q','r','s','t',
            'u','v','w','x','y','z']

#AZLyrics Link
az_link = "https://www.azlyrics.com/"

#list for urls
letter_urls = []

#Generate urls for alphabet
for letter in alphabet:
    letter_urls.append(az_link + letter + ".html")

#Stuff
artist_urls = []
artist_names = []

#Generate lists of all artists and their urls
for url in letter_urls[5:6]:
    #Collect artist names from page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    content = soup.find_all(class_="col-sm-6 text-center artist-col")

    #Fill link list and artist_names list
    for col in content:
        g_tags = col.find_all("a")
        for item in g_tags:
            artist_urls.append(az_link + item.get('href'))
            artist_names.append(item.text)
    #Sleep to avoid being blocked
    time.sleep(random.uniform(3,8))
            
#print(links, artist_names)
artist_pages=list(zip(artist_names, artist_urls))

In [ ]:
#Initialize df and name columns
with open('songInfo.csv', 'w', encoding="utf-8", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("Artist", "Album", "Year", "Song Title", "Lyrics"))
myfile.close()

In [ ]:
#Song information collection 
for item in artist_pages[:100]:
    artist_name = item[0]
    url = item[1]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    divs = soup.find_all("div")
    current_album=""
    for div in divs:
        if div.has_attr("class"):
            if div.attrs["class"][0]=="album":
                current_album = div.text
            elif div.attrs["class"][0] == "listalbum-item":
                endLink = div.a.get("href")
                songURL = "https://www.azlyrics.com/" + endLink[3:]
                curr_song = div.text
                try:
                    page1 = requests.get(songURL)
                    soupp = BeautifulSoup(page1.content, 'lxml')
                    x = soupp.find('div', {'class': 'col-xs-12 col-lg-8 text-center'})
                    y = x.find_all('div')
                    curr_year = current_album[-5:-1]
                    lyrics = y[5].text

                    try:
                        year = int(curr_year)
                        yearUP = str(year)
                        Row = []
                        Row.append(artist_name)
                        Row.append(current_album)
                        Row.append(yearUP)
                        Row.append(curr_song)
                        Row.append(artist_name)
                        Row.append(lyrics)
                        allData.append(Row)
                        print(Row[:4])
                        
                        with open('songInfo.csv', 'a', encoding="utf-8", newline='') as myfile:
                            wr = csv.writer(myfile)
                            wr.writerow(Row)
                        myfile.close()

                    except:
                        print('Invalid Year, Ignoring Song')
                except:
                    print('Could Not Collect Page Data - Probably IP Blocked')
                    
                time.sleep(random.uniform(4,16))